# Importación de librerías

In [1]:
from src.naive_median_bg_subtraction import naive_median_bg_subtraction
from src.gaussian_mix_bg_substraction import mog2_or_knn_bg_subtraction

# Experimentos

## Sustractor de fondo Naive usando la mediana

Primero se experimentará con la configuración donde el *background* calculado en base a la mediana de 30 frames aleatorios del video. Solo calculando el *background* una vez al inicio del video, sin actualizaciones entre intervalos de tiempo.

In [2]:
naive_median_bg_subtraction('vtest.avi', N=30)

The video has 795 frames and a frame rate of 10.0 frames per second.
Initial background frame calculated in 3.10 seconds
Starting background subtraction...
Average memory usage: 181.96 MB
Overall FPS: 9.97 frames/second
Average processing time: 0.001311 seconds


Se puede ver que para este caso se logró obtener una buena velocidad de procesamiento, una vez que se logró calcular el *background* inicial en aproximadamente 3 segundos. Visualmente los resultados son muy buenos, lográndose ver las máscaras correspondientes a los objetos en movimiento. Manteniendo en promedio 9.97 FPS, procesando la sustracción de fondo en 1.31 milisegundos y un uso de memoria de 181.96 MB a lo largo de la ejecución.

En el siguiente experimento se probará reduciendo el número de frames aleatorios utilizados para el cálculo del *background*, pero se incluira un intervalo de tiempo entre cada recálculo del *background*.

In [4]:
naive_median_bg_subtraction('vtest.avi', N=5, interval=10.0)

The video has 795 frames and a frame rate of 10.0 frames per second.
Initial background frame calculated in 0.52 seconds
Starting background subtraction...
Recalculating background frame at video time 00:00:10, after interval of 10.00 seconds...
Background frame recalculated on 0.4155101776123047 seconds.
Recalculating background frame at video time 00:00:20, after interval of 10.00 seconds...
Background frame recalculated on 0.3941466808319092 seconds.
Recalculating background frame at video time 00:00:30, after interval of 10.00 seconds...
Background frame recalculated on 0.3184549808502197 seconds.
Recalculating background frame at video time 00:00:40, after interval of 10.00 seconds...
Background frame recalculated on 0.469836950302124 seconds.
Recalculating background frame at video time 00:00:50, after interval of 10.00 seconds...
Background frame recalculated on 0.3358597755432129 seconds.
Recalculating background frame at video time 00:01:00, after interval of 10.00 seconds...


En este caso, se puede ver que la calidad de sustracción de fondo en general se ha reducido, llegando a mostrarse algunas máscaras que no corresponden a objetos en movimiento. Sin embargo, se logra arrancar mucho antes el procesamiento del video, además se lográ incluir un refresco del *background* cada 10 segundos. Siendo que en este caso, el tiempo de cálculo del background baja de 3 segundos aproximadamente a valores en 0.3 y 0.5 segundos. Manteniendo en promedio 9.54 FPS, procesando la sustracción de fondo en 1.18 milisegundos y un uso de memoria de 176.62 MB a lo largo de la ejecución.

Un patrón que encontraremos es que a medida que se reduzca el número de frames aleatorios utilizados para el cálculo del *background*, se irá reduciendo la calidad de la sustracción de fondo. Entonces, será necesario reducir el intervalo de tiempo entre cada recálculo del *background* para lograr mantener una buena calidad. 

El porque no se toman en cuenta más frames aleatorios para el cálculo del *background* es por el costo computacional que esto implica, ya que si se toman 30 como en el primer experimento, el tiempo de cálculo del *background* tomará un tiempo considerable cada vez que se actualice.

Entonces, un experimento útil será probar con intervalos de tiempo más largos entre cada recálculo del *background* y con un número de frames aleatorios más grande.

In [6]:
naive_median_bg_subtraction('vtest.avi', N=30, interval=90)

The video has 795 frames and a frame rate of 10.0 frames per second.
Initial background frame calculated in 3.18 seconds
Starting background subtraction...
Average memory usage: 191.20 MB
Overall FPS: 9.99 frames/second
Average processing time: 0.001249 seconds


En este caso, se logra resultados muy parecidos al del primer experimento, donde no se recalcula el *background* y se trabaja solamente con el *background* inicial. Si bien el tiempo que tomó recalcular el *background* fue considerable, este se hizo después de un tiempo mayor. Si bien para el video con el que se esta trabajando no pareciera necesario, para videos mucho más largos, el background inicial podría no ser suficiente para mantener una buena calidad a lo largo del tiempo, por lo que el recálculo del *background* será necesario.

## Sustractor de fondo utilizando algoritmos de mezcla gaussiana de OpenCV

En el siguiente experimento se prueba con el método `MOG2` de OpenCV.

In [7]:
mog2_or_knn_bg_subtraction('vtest.avi', method='MOG2')

Average memory usage: 195.70 MB
Overall FPS: 9.53 frames/second
Average processing time: 0.005439 seconds


En este caso podemos notar que visualmente se logra una buena sustracción de fondo. Sin embargo, esta no esta tan bien definida como en el caso del método naive basado en la mediana. Sin embargo, no existe un tiempo de espera inicial, ni tampoco un tiempo de espera entre cada recálculo del *background*. Logrando mantener en promedio 9.53 FPS y procesando la sustracción de fondo en 5.54 milisegundos. El uso de memoria es más alto, llegando a 195.70 MB a lo largo de la ejecución.

In [8]:
mog2_or_knn_bg_subtraction('vtest.avi', method='KNN')

Average memory usage: 189.00 MB
Overall FPS: 9.38 frames/second
Average processing time: 0.006540 seconds


Para este método se puede visualizar un funcionamiento similar al anterior. Sin embargo, en promedio se mantienen 9.38 FPS, 6.5 milisegundos para procesar la sustracción de fondo y un uso de memoria de 189 MB a lo largo de la ejecución.

---

La comparación entre los métodos de mezcla gaussiana y el método naive usando la mediana es que estos primeros tienen un funcionamiento más fluido, dado que no deben realizar el calculo del *background* inicial, ni tampoco el recálculo del *background* cada cierto intervalo de tiempo, por lo que no existirán tiempos de esperas a lo largo del procesamiento de video y podrían resultar más óptimos para videos más largos o videos transmitidos en tiempo real. 

Sin embargo, la calidad de la sustracción de fondo es superior en el método naive basado en la mediana, ya que logra una mejor definición de los objetos en movimiento.